# Text Generation using LSTM

## Topic

In this notebook I will be experimenting with an LSTM to generate text. I will be using a model text (Sherlock Holmes story) that I will train my network on to finally generate new text one charachter at a time. To do so, I will process my text first by using onehot encoding, then I will create the network and train it, finally I will generate a new charachter based on the last charachter generated by the model. So let's get started !

## Objective

- Generate new text one charachter at a time using an LSTM

### Importing Libraries

In [27]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [28]:
with open(r'C:\Users\imane\Downloads\archive (23)\text.txt', "r", encoding='utf-8') as f:
    text= f.read()

In [29]:
text = text[1255:]

In [30]:
text[:200]

' IN BOHEMIA\n\n\nI.\n\nTo Sherlock Holmes she is always _the_ woman. I have seldom heard him\nmention her under any other name. In his eyes she eclipses and\npredominates the whole of her sex. It was not tha'

So first of all I imported the text and deleted the intro to the story (Glossary and such), then I printed a sample paragraph.

In [31]:
text = text[:50000]

In [32]:
special_chars = ["&", "\n", "$", "@", '""', "—","(", "_" ]
for char in special_chars:
    text = text.replace(char, " ")

In [33]:
text[:200]

' IN BOHEMIA   I.  To Sherlock Holmes she is always  the  woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not tha'

Next I removed the special charachters to keep only the row text.

In [52]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}


encoded = np.array([char2int[ch] for ch in text])

In [53]:
encoded[:100]

array([50, 12, 14, 50, 15,  9, 23, 30, 34, 12,  7, 50, 50, 50, 12, 43, 50,
       50, 32, 24, 50,  0, 16,  3, 38,  1, 24, 10, 56, 50, 23, 24,  1, 53,
        3, 47, 50, 47, 16,  3, 50, 31, 47, 50,  8,  1, 54,  8, 26, 47, 50,
       50, 11, 16,  3, 50, 50, 54, 24, 53,  8, 20, 43, 50, 12, 50, 16,  8,
        2,  3, 50, 47,  3,  1, 33, 24, 53, 50, 16,  3,  8, 38, 33, 50, 16,
       31, 53, 50, 53,  3, 20, 11, 31, 24, 20, 50, 16,  3, 38, 50])

In the above I encoded the characters by giving them each a unique number after of course splitting them, then I created two dictinaries one that goves the unique number of each character and the other that gives a the character associated to each number.

In [54]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [55]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make 
    char_per_batch = batch_size * seq_length
    n_batches = len(arr)//char_per_batch
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:char_per_batch*n_batches]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape(batch_size, -1)
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] =  x[:, 1:] , arr[:, 0]
        yield x, y

The function get batches takes in the encoded characters and returns batches of them where the x is a sequence of numbers and the y is the same sequence shifted by 1.

In [56]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available')

No GPU available


In [57]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, dropout = drop_prob, batch_first = True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, n_hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        out = out.reshape(-1, self.n_hidden)
        out = self.fc(out)
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

The class CharRNN is where I created my model, it contains a stacked lstm layer, a dropout layer and a fully connexted layers with a hidden and cell states.

In [88]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):

    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss)
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.average(val_losses)))

The training function specifies a loss criterion and optimizer first, then it iterates through the batches, passing every input through the network and calculating the loss based on the output and actual label. Then the validation step goes through the same process to produce a validation loss.

In [89]:
n_hidden= 512
n_layers= 2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(67, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=67, bias=True)
)


In [92]:
batch_size = 128
seq_length = 100
n_epochs = 200

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 4/200... Step: 10... Loss: 1.7791... Val Loss: nan
Epoch: 7/200... Step: 20... Loss: 1.7315... Val Loss: nan
Epoch: 10/200... Step: 30... Loss: 1.7415... Val Loss: nan
Epoch: 14/200... Step: 40... Loss: 1.6750... Val Loss: nan
Epoch: 17/200... Step: 50... Loss: 1.6495... Val Loss: nan
Epoch: 20/200... Step: 60... Loss: 1.6609... Val Loss: nan
Epoch: 24/200... Step: 70... Loss: 1.6082... Val Loss: nan
Epoch: 27/200... Step: 80... Loss: 1.5694... Val Loss: nan
Epoch: 30/200... Step: 90... Loss: 1.5902... Val Loss: nan
Epoch: 34/200... Step: 100... Loss: 1.5362... Val Loss: nan
Epoch: 37/200... Step: 110... Loss: 1.4924... Val Loss: nan
Epoch: 40/200... Step: 120... Loss: 1.5085... Val Loss: nan
Epoch: 44/200... Step: 130... Loss: 1.4582... Val Loss: nan
Epoch: 47/200... Step: 140... Loss: 1.4157... Val Loss: nan
Epoch: 50/200... Step: 150... Loss: 1.4152... Val Loss: nan
Epoch: 54/200... Step: 160... Loss: 1.3540... Val Loss: nan
Epoch: 57/200... Step: 170... Loss: 1.3153... Val L

The training happened over 200 epochs, through which the training loss decreses to reach .35, unfortunately I cannot see the testing loss (I've tried to solve the issue but couldn't)

In [93]:
model_name = 'lstm_200_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

Then I saved the latest model with all its parameters.

In [94]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

The predict function takes in a character and predicts the next character by randomly picking through the topk characters i.e the characters with the highest probability distribution.

In [95]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

Then the sample function starts off with a prime word, takes as input a character and produces the next one to be also used as input in the next step.

In [100]:
print(sample(net, 1000, prime='Hello', top_k=3))

Hellonoro ale therered he alalerorare harale he hed are as halarerarere the th there th heras he as t as t h thare arerere h halalinono there therored has arare hale thas thare aro has t thared th thered as thaline tharerererered as are as he arore arone as thalin hererereras he thararare the thed the are t he therered hererero are aleraro hereronerererase t thale he arerase there ale to ton hereredo halaroras thas aredo t hare thalarere t thare he harerored alito are t thed hed arored the torored tore ase h therereras halis. hale the h th aras t arero has the t has arederere t the harerered aronore he alin alarasedoras tharo there h aralas harerero has the thaled torered arerorere h th haredo the heredere as alithared arerarere arererede haras he the h he t aled alereroras aliserero here hale thalas heded thered haras he thas arered torerarered hare tharererered thered toreras herere th t alaro alared hed are he h as alerororered t to to aled alithared tharerere to ale has ared asere 

In [97]:
with open('lstm_200_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [101]:
print(sample(loaded, 2000, top_k=3, prime="And Sherlock said"))

And Sherlock said haro the has alinerono hareras ale alarerored areded as h h as arerereras arered alas thas he as alin hed he arererere he thareras h to are t he tharered haleron t tharonerere the halithale here here herared there to h ale tono t hale tharere hero arerere he as therere torerere halin halere to hed he harerererered torored alalerere he alithed th he as are t to aro to ale here hed has hed aserered the as the he hed he are to harerere he therasererererererorerere as the he he as t are thas. as h hare thed as heded as asere he thare here arereredore alared aleralerare hare hed therero aras ton aro he th hed thared are therere has haled h heralin as he the h ared the thale the as hede the torere tore has thered hererere as he h arerarero the th here herere ared thed hase aron thalis heralered harerere h the hed areraserere tharororarede he halere asedon hedorere thede herore t harono as arere ased thererere thed thalasered the as the as hed are herereras t hered as. thed 

Finally I tried to generate some text, the results are not very good seen as the text generated doesn't make much sense.